# ExperimentLGR
### Take the dataset where the negatives comprise of random genomic data (including repeats) sampled for length and gc content.
### Train a single separable convolutional classifier on triplets and evaluate

## Imports

In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
# # Credit: https://stackoverflow.com/questions/34199233/how-to-prevent-tensorflow-from-allocating-the-totality-of-a-gpu-memory
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [7]:
import data
from tensorflow import keras
import nets
from loaders import PremadeTripletClassifierSequence

## Files

In [8]:
# file_name = '/home/zakaria/Data/IdentityData/ShortTrain/60.fa'#file_name = '../Data/sequences.fa'

data_dir = '../Data/Datasets/LGR/'


train_sequence = f'{data_dir}/train_sequences.fa'
valid_sequence = f'{data_dir}/valid_sequences.fa'
test_sequence = f'{data_dir}/test_sequences.fa'

reverse_train_sequence = f'{data_dir}/reverse_train_sequences.fa'
reverse_valid_sequence = f'{data_dir}/reverse_valid_sequences.fa'
reverse_test_sequence = f'{data_dir}/reverse_test_sequences.fa'

train_triplet_dis = f'{data_dir}/train_triplet_dis.npy'
valid_triplet_dis = f'{data_dir}/valid_triplet_dis.npy'
test_triplet_dis = f'{data_dir}/test_triplet_dis.npy'

train_triplet_sim = f'{data_dir}/train_triplet_sim.npy'
valid_triplet_sim = f'{data_dir}/valid_triplet_sim.npy'
test_triplet_sim = f'{data_dir}/test_triplet_sim.npy'

## Parameters

In [9]:
codings_size=100 
exp_filter_num=4
exp_filter_1d_size= 11

allow_reverse = True
max_len = 600

learning_rate = 0.1
momentum = 0.9
is_nesterov = True
patience = 20
monitor = 'val_crm_f1_score'
start_from_epoch = 10

## Loading Data

In [10]:
train = data.FantomToOneHotConverter(train_sequence, 0, max_len).seq_matrix
reverse_train = data.FantomToOneHotConverter(reverse_train_sequence, 0, max_len).seq_matrix if allow_reverse else None

In [11]:
valid = data.FantomToOneHotConverter(valid_sequence, 0, max_len).seq_matrix
reverse_valid = data.FantomToOneHotConverter(reverse_valid_sequence, 0, max_len).seq_matrix if allow_reverse else None

In [12]:
# test = data.FantomToOneHotConverter(test_sequence, 0, max_len).seq_matrix
# reverse_test = data.FantomToOneHotConverter(reverse_test_sequence, 0, max_len).seq_matrix if allow_reverse else None

In [13]:
train_seq = PremadeTripletClassifierSequence(train, train_triplet_sim, train_triplet_dis, batch_size = 1024, reverse_x_in =reverse_train)
valid_seq = PremadeTripletClassifierSequence(valid, valid_triplet_sim, valid_triplet_dis, batch_size = 1024, reverse_x_in =reverse_valid)
# test_seq = PremadeTripletClassifierSequence(test, test_triplet_sim, test_triplet_dis, batch_size = 1024, reverse_x_in =reverse_test)

In [14]:
for x_batch, y_batch in train_seq:
    _, d1, d2, d3 = x_batch.shape

## Creating Model

In [15]:
model = nets.make_conv_classifier(codings_size, (d1, d2, d3), exp_filter_1d_size, filter_num=exp_filter_num)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4, 600, 3)]       0         
                                                                 
 masking (Masking)           (None, 4, 600, 3)         0         
                                                                 
 separable_conv2d (Separable  (None, 1, 598, 4)        48        
 Conv2D)                                                         
                                                                 
 batch_normalization (BatchN  (None, 1, 598, 4)        16        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 1, 598, 4)         0         
                                                                 
 reshape (Reshape)           (None, 598, 4)            0     

In [16]:
opt = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=is_nesterov) # 0.1
model.compile(loss='mse', metrics=['accuracy', nets.crm_specificity, tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), nets.crm_f1_score], optimizer=opt) #  

In [17]:
early_stopping = keras.callbacks.EarlyStopping(patience=patience, min_delta=1/100000, restore_best_weights=True, monitor=monitor, start_from_epoch = start_from_epoch, mode='max') #val_precision

In [18]:
def exponential_decay_fn(epoch, lr):
    '''
    This function decreases the learning rate according to the epoch
    '''
    return lr*0.1**(1/100)

lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

## Training Model

In [19]:
model.fit(train_seq, epochs=500, validation_data=valid_seq, workers=26, callbacks=[early_stopping, lr_scheduler]) #,workers=26

Epoch 1/500
72/72 [==============================] - 16s 167ms/step - loss: 0.2426 - accuracy: 0.5785 - crm_specificity: 0.6408 - recall: 0.5165 - precision: 0.5896 - crm_f1_score: 0.5504 - val_loss: 0.2461 - val_accuracy: 0.5099 - val_crm_specificity: 0.0274 - val_recall: 0.9924 - val_precision: 0.5050 - val_crm_f1_score: 0.6693 - lr: 0.0977
Epoch 2/500
72/72 [==============================] - 11s 155ms/step - loss: 0.2289 - accuracy: 0.6122 - crm_specificity: 0.6567 - recall: 0.5677 - precision: 0.6230 - crm_f1_score: 0.5940 - val_loss: 0.2347 - val_accuracy: 0.6011 - val_crm_specificity: 0.7912 - val_recall: 0.4109 - val_precision: 0.6630 - val_crm_f1_score: 0.5071 - lr: 0.0955
Epoch 3/500
72/72 [==============================] - 11s 157ms/step - loss: 0.2242 - accuracy: 0.6289 - crm_specificity: 0.6500 - recall: 0.6078 - precision: 0.6345 - crm_f1_score: 0.6207 - val_loss: 0.2265 - val_accuracy: 0.6233 - val_crm_specificity: 0.6880 - val_recall: 0.5584 - val_precision: 0.6405 - val

In [ ]:
model.save("ExperimentLGRModel")

## Evaluating Model

In [20]:
nets.print_results(train_seq, model)

72/72 [==============================] - 9s 122ms/step - loss: 0.2037 - accuracy: 0.6812 - crm_specificity: 0.4813 - recall: 0.8812 - precision: 0.6294 - crm_f1_score: 0.7342
0.2033,68.19,47.93,88.47,62.95,73.54


In [21]:
nets.print_results(valid_seq, model)

15/15 [==============================] - 2s 122ms/step - loss: 0.2086 - accuracy: 0.6705 - crm_specificity: 0.4887 - recall: 0.8526 - precision: 0.6247 - crm_f1_score: 0.7209
0.2089,67.23,48.32,86.15,62.49,72.42
